### Chat with RAG

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.environ["GROQ_API_KEY"]

In [6]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings

In [4]:
model = ChatGroq(
    model="llama3-8b-8192"
)

In [5]:
model.invoke("Hello how are you.")

AIMessage(content="Hello! I'm just an AI, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 15, 'total_tokens': 61, 'completion_time': 0.03563343, 'prompt_time': 0.003603488, 'queue_time': None, 'total_time': 0.039236918}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-0ae32196-d2f7-4a54-ad5d-3a20920a932b-0')

In [8]:
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-MiniLM-L6-v2"
)

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
os.environ["PINECONE_API_KEY"] = os.environ["PINECONE_API_KEY"]
index_name = "books-pinecone-with-metadata"

vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embedding
)

In [12]:
vectorstore.similarity_search("What is Langchain?")

[Document(page_content='LangChain: A Framework for LLM-Powered Applications\nLangChain is a powerful and flexible framework designed to simplify the development of applications that harness the capabilities of large language models (LLMs). It provides a wide range of tools, abstractions, and integrations that help developers build, customize, and optimize applications that leverage LLMs for tasks like text generation, question answering, summarization, chatbots, and more.', metadata={'source': 'langchain_demo.txt'}),
 Document(page_content='Key Features and Benefits\nModular Components: LangChain offers a variety of modular components (chains, agents, tools, prompts, memory, etc.) that can be easily combined and customized to build complex LLM-powered workflows.\nData Integration: It seamlessly integrates with various data sources, enabling applications to access and process external information, enhancing the context and relevance of LLM responses.\nAgent Frameworks: LangChain provide

In [15]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 2
    }
)

relevant_docs = retriever.invoke("What is langchain library")

In [17]:
for i, doc in enumerate(relevant_docs):
    print(f"Document {i+1}: {doc.page_content}")
    print("\n")

Document 1: LangChain: A Framework for LLM-Powered Applications
LangChain is a powerful and flexible framework designed to simplify the development of applications that harness the capabilities of large language models (LLMs). It provides a wide range of tools, abstractions, and integrations that help developers build, customize, and optimize applications that leverage LLMs for tasks like text generation, question answering, summarization, chatbots, and more.


Document 2: Key Features and Benefits
Modular Components: LangChain offers a variety of modular components (chains, agents, tools, prompts, memory, etc.) that can be easily combined and customized to build complex LLM-powered workflows.
Data Integration: It seamlessly integrates with various data sources, enabling applications to access and process external information, enhancing the context and relevance of LLM responses.
Agent Frameworks: LangChain provides agent frameworks that allow LLMs to interact with their environment, m

In [18]:
print("".join([doc.page_content for doc in relevant_docs]))

LangChain: A Framework for LLM-Powered Applications
LangChain is a powerful and flexible framework designed to simplify the development of applications that harness the capabilities of large language models (LLMs). It provides a wide range of tools, abstractions, and integrations that help developers build, customize, and optimize applications that leverage LLMs for tasks like text generation, question answering, summarization, chatbots, and more.Key Features and Benefits
Modular Components: LangChain offers a variety of modular components (chains, agents, tools, prompts, memory, etc.) that can be easily combined and customized to build complex LLM-powered workflows.
Data Integration: It seamlessly integrates with various data sources, enabling applications to access and process external information, enhancing the context and relevance of LLM responses.
Agent Frameworks: LangChain provides agent frameworks that allow LLMs to interact with their environment, make decisions, and take act

In [34]:
query = "what is langchain?"

combined_input = (
    "Here are some documents that might help answer the question: "
    + query
    + "\some information:\n"
    + "\n".join([doc.page_content for doc in relevant_docs])
    + "\n answer the question, just answer dont say anything about accorind to the documents or anything like that"
)

<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
C:\Users\acer\AppData\Local\Temp\ipykernel_16148\4139268699.py:6: SyntaxWarning: invalid escape sequence '\s'
  + "\some information:\n"


In [35]:
print(combined_input)

Here are some documents that might help answer the question: what is langchain?\some information:
LangChain: A Framework for LLM-Powered Applications
LangChain is a powerful and flexible framework designed to simplify the development of applications that harness the capabilities of large language models (LLMs). It provides a wide range of tools, abstractions, and integrations that help developers build, customize, and optimize applications that leverage LLMs for tasks like text generation, question answering, summarization, chatbots, and more.
Key Features and Benefits
Modular Components: LangChain offers a variety of modular components (chains, agents, tools, prompts, memory, etc.) that can be easily combined and customized to build complex LLM-powered workflows.
Data Integration: It seamlessly integrates with various data sources, enabling applications to access and process external information, enhancing the context and relevance of LLM responses.
Agent Frameworks: LangChain provide

In [36]:
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=combined_input)
]

In [37]:
result = model.invoke(messages)
print(result.content)

LangChain is a framework designed to simplify the development of applications that use large language models (LLMs).
